# * VINSIGHT : Subscriber
    กลุ่ม KPIs ของ Postpaid Reported SubBase B2C : TRUE มาจากพี่เอ้รัน scripts พี่หนุ่ย (SubBase, 60DPD, Gain/Loss 60DPD)
    กลุ่ม KPIs ของ Postpaid Reported SubBase B2C : DTAC ยังมาจากพี่เอกรันให้อยู่ (SubBase, 60DPD, Gain/Loss 60DPD)
    กลุ่ม KPIs ของ FTTx Reported SubBase ยังมาจากพี่เอกรันให้อยู่ (SubBase, 60DPD, Gain/Loss 60DPD)
    Postpaid Reported SubBase B2C เริ่มตั้งแต่ June2024++
    Prepaid Active Caller : TMH เริ่มตั้งแต่ May2024++

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000500' --Prepaid Usage Subs
        , 'TB1S000500' --Prepaid Usage Subs : TMH
        , 'DB1S000500' --Prepaid Usage Subs : DTAC
        , 'B1S000600' --Prepaid Active Caller 30D Rolling
        , 'TB1S000600' --Prepaid Active Caller 30D Rolling : TMH
        , 'DB1S000600' --Prepaid Active Caller 30D Rolling : DTAC
        , 'B1S000702' --Prepaid SubBase
        , 'DB1S000702' --Prepaid SubBase : DTAC
        , 'TB1S000702' --Prepaid SubBase : TMH
        , 'B1S000700' --Prepaid Reported SubBase
        , 'DB1S000700' --Prepaid Reported SubBase : DTAC
        , 'TB1S000700' --Prepaid Reported SubBase : TMH
        
        , 'B2S010602' --Postpaid SubBase B2C
        , 'DB2S010602' --Postpaid SubBase B2C : DTAC
        , 'TB2S010602' --Postpaid SubBase B2C : TMH
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'DB2S010600' --Postpaid Reported SubBase B2C : DTAC
        , 'TB2S010600' --Postpaid Reported SubBase B2C : TMH
        
        , 'TB3S020604' --FTTx SubBase
        , 'TB3S000600' --FTTx Reported SubBase

        , 'TB4S000500' --TVS Active Subs
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
    --ORDER BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-10-27, 12:42:53

DataFrame: 10640 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

## Overview by Period

In [4]:
''' Subscriber Yearly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
sub_yearly_df = sub_yearly_df.loc[sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

sub_yearly_df['PRE_CALL_30D'] = np.where(sub_yearly_df['METRIC_CD']=='B1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['PRE_CALL_30D_T'] = np.where(sub_yearly_df['METRIC_CD']=='TB1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['PRE_CALL_30D_D'] = np.where(sub_yearly_df['METRIC_CD']=='DB1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_SUB_B2C'] = np.where(sub_yearly_df['METRIC_CD']=='B2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_yearly_df['METRIC_CD']=='TB2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_yearly_df['METRIC_CD']=='DB2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['TOL_RPT_SUB'] = np.where(sub_yearly_df['METRIC_CD']=='TB3S000600', sub_yearly_df['P'], 0)
sub_yearly_df['TVS_ACTV_SUB'] = np.where(sub_yearly_df['METRIC_CD']=='TB4S000500', sub_yearly_df['P'], 0)

sub_yearly_df = sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_yearly_df = sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_yearly_df_display = sub_yearly_df.copy()
sub_yearly_df_display['ACTUAL_AS_OF'] = sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_yearly_df_display[col] = sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,2024,2025-09-22 06:15:44,20241231,"28,100,141","17,769,553","10,330,588","14,158,698","9,102,725","5,055,973","3,154,889","1,180,410"
1,2025,2025-10-27 06:22:35,20251025,"25,681,708","16,025,537","9,656,171","13,940,780","9,051,017","4,879,958","3,232,773","1,064,471"


In [5]:
''' Subscriber Quarterly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_QTR
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

sub_quarterly_df['PRE_CALL_30D'] = np.where(sub_quarterly_df['METRIC_CD']=='B1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['PRE_CALL_30D_T'] = np.where(sub_quarterly_df['METRIC_CD']=='TB1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['PRE_CALL_30D_D'] = np.where(sub_quarterly_df['METRIC_CD']=='DB1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_SUB_B2C'] = np.where(sub_quarterly_df['METRIC_CD']=='B2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_quarterly_df['METRIC_CD']=='TB2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_quarterly_df['METRIC_CD']=='DB2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TOL_RPT_SUB'] = np.where(sub_quarterly_df['METRIC_CD']=='TB3S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TVS_ACTV_SUB'] = np.where(sub_quarterly_df['METRIC_CD']=='TB4S000500', sub_quarterly_df['P'], 0)

sub_quarterly_df = sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_quarterly_df = sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_quarterly_df_display = sub_quarterly_df.copy()
sub_quarterly_df_display['ACTUAL_AS_OF'] = sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_quarterly_df_display[col] = sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,20241,2025-09-22 06:15:44,20240331,0,0,"10,558,566",0,"8,994,853",0,"3,090,287","1,258,065"
1,20242,2025-09-22 06:15:44,20240630,"27,800,517","17,313,488","10,487,029","14,226,167","9,042,184","5,183,983","3,107,084","1,236,606"
2,20243,2025-09-22 06:15:44,20240930,"27,505,003","17,190,424","10,314,579","14,186,615","9,068,470","5,118,145","3,127,304","1,205,116"
3,20244,2025-09-22 06:15:44,20241231,"28,100,141","17,769,553","10,330,588","14,158,698","9,102,725","5,055,973","3,154,889","1,180,410"
4,20251,2025-10-27 06:22:35,20250331,"27,140,083","17,215,465","9,924,618","14,084,812","9,086,820","4,997,992","3,187,717","1,153,137"
5,20252,2025-10-27 06:22:35,20250630,"26,272,750","16,372,729","9,900,021","14,036,540","9,084,007","4,952,533","3,207,941","1,112,829"
6,20253,2025-10-27 06:22:35,20250930,"25,525,284","15,912,532","9,612,752","13,952,680","9,061,049","4,891,631","3,225,844","1,071,847"
7,20254,2025-10-27 06:22:35,20251025,"25,681,708","16,025,537","9,656,171","13,940,780","9,051,017","4,879,958","3,232,773","1,064,471"


In [6]:
''' Subscriber Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr]
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

sub_monthly_df['PRE_CALL_30D'] = np.where(sub_monthly_df['METRIC_CD']=='B1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['PRE_CALL_30D_T'] = np.where(sub_monthly_df['METRIC_CD']=='TB1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['PRE_CALL_30D_D'] = np.where(sub_monthly_df['METRIC_CD']=='DB1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_SUB_B2C'] = np.where(sub_monthly_df['METRIC_CD']=='B2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_monthly_df['METRIC_CD']=='TB2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_monthly_df['METRIC_CD']=='DB2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['TOL_RPT_SUB'] = np.where(sub_monthly_df['METRIC_CD']=='TB3S000600', sub_monthly_df['P'], 0)
sub_monthly_df['TVS_ACTV_SUB'] = np.where(sub_monthly_df['METRIC_CD']=='TB4S000500', sub_monthly_df['P'], 0)

sub_monthly_df = sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_monthly_df = sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_monthly_df_display = sub_monthly_df.copy()
sub_monthly_df_display['ACTUAL_AS_OF'] = sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_monthly_df_display[col] = sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,202401,2025-09-22 06:15:44,20240131,0,0,"10,612,127",0,0,0,0,"1,280,972"
1,202402,2025-09-22 06:15:44,20240229,0,0,"10,729,197",0,"8,998,482",0,"3,084,265","1,269,692"
2,202403,2025-09-22 06:15:44,20240331,0,0,"10,558,566",0,"8,994,853",0,"3,090,287","1,258,065"
3,202404,2025-09-22 06:15:44,20240430,0,0,"10,637,624",0,"8,996,646",0,"3,091,767","1,249,008"
4,202405,2025-09-22 06:15:44,20240531,"28,022,052","17,391,242","10,630,810",0,"9,023,324",0,"3,098,514","1,245,639"
5,202406,2025-09-22 06:15:44,20240630,"27,800,517","17,313,488","10,487,029","14,226,167","9,042,184","5,183,983","3,107,084","1,236,606"
6,202407,2025-09-22 06:15:44,20240731,"27,805,282","17,342,927","10,462,355","14,209,033","9,052,569","5,156,464","3,114,835","1,226,581"
7,202408,2025-09-22 06:15:44,20240831,"27,664,960","17,247,805","10,417,155","14,190,431","9,061,002","5,129,429","3,122,029","1,214,582"
8,202409,2025-09-22 06:15:44,20240930,"27,505,003","17,190,424","10,314,579","14,186,615","9,068,470","5,118,145","3,127,304","1,205,116"
9,202410,2025-09-22 06:15:44,20241031,"27,425,422","17,195,820","10,229,602","14,179,406","9,080,143","5,099,263","3,134,367","1,195,552"


In [7]:
''' Subscriber Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_WK
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=prev_mth]
sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

sub_weekly_df['PRE_CALL_30D'] = np.where(sub_weekly_df['METRIC_CD']=='B1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['PRE_CALL_30D_T'] = np.where(sub_weekly_df['METRIC_CD']=='TB1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['PRE_CALL_30D_D'] = np.where(sub_weekly_df['METRIC_CD']=='DB1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_SUB_B2C'] = np.where(sub_weekly_df['METRIC_CD']=='B2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_weekly_df['METRIC_CD']=='TB2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_weekly_df['METRIC_CD']=='DB2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['TOL_RPT_SUB'] = np.where(sub_weekly_df['METRIC_CD']=='TB3S000600', sub_weekly_df['P'], 0)
sub_weekly_df['TVS_ACTV_SUB'] = np.where(sub_weekly_df['METRIC_CD']=='TB4S000500', sub_weekly_df['P'], 0)

sub_weekly_df = sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_weekly_df = sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_weekly_df_display = sub_weekly_df.copy()
sub_weekly_df_display['ACTUAL_AS_OF'] = sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_weekly_df_display[col] = sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,2025036,2025-10-27 06:22:35,20250907,"25,566,281","15,922,426","9,643,855","13,976,943","9,069,935","4,907,008","3,217,996","1,079,331"
1,2025037,2025-10-27 06:22:35,20250914,"25,542,937","15,917,341","9,625,596","13,959,714","9,060,333","4,899,381","3,216,381","1,076,159"
2,2025038,2025-10-27 06:22:35,20250921,"25,515,797","15,907,850","9,607,947","13,956,235","9,059,509","4,896,726","3,218,675","1,073,528"
3,2025039,2025-10-27 06:22:35,20250928,"25,509,673","15,909,170","9,600,503","13,945,044","9,055,440","4,889,604","3,219,769","1,072,500"
4,2025040,2025-10-27 06:22:35,20251005,"25,542,856","15,941,685","9,601,171","13,953,938","9,063,722","4,890,216","3,227,049","1,070,257"
5,2025041,2025-10-27 06:22:35,20251012,"25,553,333","15,952,772","9,600,561","13,937,719","9,053,935","4,883,784","3,226,523","1,067,968"
6,2025042,2025-10-27 06:22:35,20251019,"25,640,851","15,999,259","9,641,592","13,929,703","9,055,687","4,874,016","3,229,159","1,065,355"
7,2025043,2025-10-27 06:22:35,20251025,"25,681,708","16,025,537","9,656,171","13,940,780","9,051,017","4,879,958","3,232,773","1,064,471"


In [8]:
''' Subscriber Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
sub_daily_df = sub_daily_df.loc[sub_daily_df['METRIC_CD'].isin(v_metric_list)]

sub_daily_df['PRE_CALL_30D'] = np.where(sub_daily_df['METRIC_CD']=='B1S000600', sub_daily_df['P'], 0)
sub_daily_df['PRE_CALL_30D_T'] = np.where(sub_daily_df['METRIC_CD']=='TB1S000600', sub_daily_df['P'], 0)
sub_daily_df['PRE_CALL_30D_D'] = np.where(sub_daily_df['METRIC_CD']=='DB1S000600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C'] = np.where(sub_daily_df['METRIC_CD']=='B2S010600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C_T'] = np.where(sub_daily_df['METRIC_CD']=='TB2S010600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C_D'] = np.where(sub_daily_df['METRIC_CD']=='DB2S010600', sub_daily_df['P'], 0)
sub_daily_df['TOL_RPT_SUB'] = np.where(sub_daily_df['METRIC_CD']=='TB3S000600', sub_daily_df['P'], 0)
sub_daily_df['TVS_ACTV_SUB'] = np.where(sub_daily_df['METRIC_CD']=='TB4S000500', sub_daily_df['P'], 0)

sub_daily_df = sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_daily_df = sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
sub_daily_df = sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_daily_df_display = sub_daily_df.copy()
mod_col_list = sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_daily_df_display[col] = sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,202510,20251026,2025-10-27 06:22:35,0,0,0,0,0,0,0,0
1,202510,20251025,2025-10-27 06:22:35,"25,681,708","16,025,537","9,656,171",0,"9,051,017",0,0,"1,064,471"
2,202510,20251024,2025-10-27 06:22:35,"25,693,650","16,031,660","9,661,990","13,940,780","9,060,822","4,879,958","3,232,773","1,064,809"
3,202510,20251023,2025-10-27 06:22:35,"25,680,129","16,030,226","9,649,903","13,936,785","9,058,101","4,878,684","3,231,636","1,065,047"
4,202510,20251022,2025-10-27 06:22:35,"25,692,501","16,028,672","9,663,829","13,932,529","9,055,329","4,877,200","3,230,173","1,065,539"
5,202510,20251021,2025-10-27 06:22:35,"25,697,974","16,023,578","9,674,396","13,929,213","9,053,028","4,876,185","3,228,301","1,065,064"
6,202510,20251020,2025-10-27 06:22:35,"25,675,340","16,019,580","9,655,760","13,934,890","9,059,147","4,875,743","3,231,090","1,065,064"
7,202510,20251019,2025-10-27 06:22:35,"25,640,851","15,999,259","9,641,592","13,929,703","9,055,687","4,874,016","3,229,159","1,065,355"
8,202510,20251018,2025-10-27 06:22:35,"25,602,716","15,967,596","9,635,120","13,925,006","9,052,205","4,872,801","3,227,333","1,065,931"
9,202510,20251017,2025-10-27 06:22:35,"25,602,814","15,974,402","9,628,412","13,938,946","9,057,982","4,880,964","3,230,334","1,066,108"


## Products Summary

### Prep Monthly Data

In [9]:
''' Monthly Summary '''

monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
monthly_df_display['ACTUAL_AS_OF'] = monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [10]:
''' Parameter '''

# v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
v_tm_key_mth = 202501

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202501


### Prepaid

In [11]:
''' Prepaid Usage Subs '''

v_metric_list = [
	'B1S000500' #Prepaid Usage Subs
    , 'TB1S000500' #Prepaid Usage Subs : TMH
    , 'DB1S000500' #Prepaid Usage Subs : DTAC
    ]

prepaid_usage_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_usage_sub_df = prepaid_usage_sub_df.loc[prepaid_usage_sub_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_usage_sub_df = prepaid_usage_sub_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
# prepaid_usage_sub_df = prepaid_usage_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_usage_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [12]:
''' Prepaid Active Caller 30D Rolling '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    ]

prepaid_call_30d_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_call_30d_df = prepaid_call_30d_df.loc[prepaid_call_30d_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_call_30d_df = prepaid_call_30d_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_call_30d_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Prepaid,B1S000600,Prepaid Active Caller 30D Rolling,2025-10-27 06:22:35,20250131,"27,745,939","27,745,939","27,731,832","27,731,832","27,731,832",0
1,202501,Prepaid,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-10-27 06:22:35,20250131,"10,401,645","10,401,645","10,400,923","10,400,923","10,400,923",0
2,202501,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,20250131,"17,344,294","17,344,294","17,330,909","17,330,909","17,330,909",0
3,202502,Prepaid,B1S000600,Prepaid Active Caller 30D Rolling,2025-10-27 06:22:35,20250228,"27,605,672","27,605,672","27,587,202","27,587,202","27,587,202",0
4,202502,Prepaid,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-10-27 06:22:35,20250228,"10,180,996","10,180,996","10,179,777","10,179,777","10,179,777",0
5,202502,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,20250228,"17,424,676","17,424,676","17,407,425","17,407,425","17,407,425",0
6,202503,Prepaid,B1S000600,Prepaid Active Caller 30D Rolling,2025-10-27 06:22:35,20250331,"27,140,083","27,140,083","27,119,364","27,119,364","27,119,364",0
7,202503,Prepaid,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-10-27 06:22:35,20250331,"9,924,618","9,924,618","9,923,719","9,923,719","9,923,719",0
8,202503,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,20250331,"17,215,465","17,215,465","17,195,645","17,195,645","17,195,645",0
9,202504,Prepaid,B1S000600,Prepaid Active Caller 30D Rolling,2025-10-27 06:22:35,20250430,"26,599,795","26,599,795","26,578,640","26,578,640","26,578,640",0


In [13]:
''' Prepaid SubBase '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
    ]

prepaid_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_subbase_df = prepaid_subbase_df.loc[prepaid_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_subbase_df = prepaid_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20250131,0,"24,942,269","24,769,264","24,267,797","24,769,149","-24,942,269"
1,202502,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20250228,0,"24,921,942","24,760,083","24,758,973","24,758,973","-24,921,942"
2,202503,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20250331,0,"24,578,672","24,437,486","24,436,544","24,436,544","-24,578,672"
3,202504,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20250430,0,"23,721,266","23,582,316","23,581,524","23,581,524","-23,721,266"
4,202505,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20250531,0,"23,721,160","23,561,656","23,560,980","23,560,980","-23,721,160"
5,202506,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20250630,0,"22,959,730","22,840,094","22,839,500","22,839,500","-22,959,730"
6,202507,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20250731,0,"22,320,765","22,224,915","22,224,384","22,224,384","-22,320,765"
7,202508,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20250831,0,"21,571,836","21,490,978","21,490,494","21,490,494","-21,571,836"
8,202509,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20250930,0,"21,760,938","21,627,554","21,627,099","21,627,099","-21,760,938"
9,202510,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-10-27 06:22:35,20251024,0,"21,961,303","21,828,397","21,827,959","21,827,959","-21,961,303"


In [14]:
''' Prepaid Reported SubBase '''

v_metric_list = [
    'B1S000700' #Prepaid Reported SubBase
    , 'DB1S000700' #Prepaid Reported SubBase : DTAC
    , 'TB1S000700' #Prepaid Reported SubBase : TMH
    ]

prepaid_report_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_report_sub_df = prepaid_report_sub_df.loc[prepaid_report_sub_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_report_sub_df = prepaid_report_sub_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_report_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Prepaid,B1S000700,Prepaid Reported SubBase,2025-10-27 06:22:35,20250131,0,"34,864,771","34,730,790","34,730,790","34,730,790","-34,864,771"
1,202501,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-10-27 06:22:35,20250131,0,"13,485,316","13,468,594","13,468,594","13,468,594","-13,485,316"
2,202501,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-10-27 06:22:35,20250131,0,"21,379,455","21,262,196","21,262,196","21,262,196","-21,379,455"
3,202502,Prepaid,B1S000700,Prepaid Reported SubBase,2025-10-27 06:22:35,20250228,0,"34,357,878","34,245,256","34,245,256","34,245,256","-34,357,878"
4,202502,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-10-27 06:22:35,20250228,0,"13,187,568","13,166,731","13,166,731","13,166,731","-13,187,568"
5,202502,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-10-27 06:22:35,20250228,0,"21,170,310","21,078,525","21,078,525","21,078,525","-21,170,310"
6,202503,Prepaid,B1S000700,Prepaid Reported SubBase,2025-10-27 06:22:35,20250331,0,"33,574,556","33,467,036","33,467,036","33,467,036","-33,574,556"
7,202503,Prepaid,DB1S000700,Prepaid Reported SubBase : DTAC,2025-10-27 06:22:35,20250331,0,"12,927,387","12,907,244","12,907,244","12,907,244","-12,927,387"
8,202503,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-10-27 06:22:35,20250331,0,"20,647,169","20,559,792","20,559,792","20,559,792","-20,647,169"
9,202504,Prepaid,B1S000700,Prepaid Reported SubBase,2025-10-27 06:22:35,20250430,0,"33,077,534","32,963,110","32,963,110","32,963,110","-33,077,534"


### Postpaid

In [15]:
''' Postpaid SubBase B2C '''

v_metric_list = [
    'B2S010602' #Postpaid SubBase B2C
    , 'DB2S010602' #Postpaid SubBase B2C : DTAC
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
    ]

postpaid_subbase_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.loc[postpaid_subbase_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_subbase_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Postpaid,B2S010602,Postpaid SubBase B2C,2025-10-27 06:22:35,20250131,0,"14,274,718","14,268,821","14,092,567","14,268,794","-14,274,718"
1,202501,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-10-27 06:22:35,20250131,0,"5,161,625","5,155,964","4,979,737","5,155,964","-5,161,625"
2,202501,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,20250131,0,"9,113,093","9,112,857","9,112,830","9,112,830","-9,113,093"
3,202502,Postpaid,B2S010602,Postpaid SubBase B2C,2025-10-27 06:22:35,20250228,0,"14,273,218","14,267,309","14,267,293","14,267,293","-14,273,218"
4,202502,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-10-27 06:22:35,20250228,0,"5,138,667","5,133,010","5,133,010","5,133,010","-5,138,667"
5,202502,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,20250228,0,"9,134,551","9,134,299","9,134,283","9,134,283","-9,134,551"
6,202503,Postpaid,B2S010602,Postpaid SubBase B2C,2025-10-27 06:22:35,20250331,0,"14,288,650","14,282,740","14,282,728","14,282,728","-14,288,650"
7,202503,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-10-27 06:22:35,20250331,0,"5,118,045","5,112,387","5,112,387","5,112,387","-5,118,045"
8,202503,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-10-27 06:22:35,20250331,0,"9,170,605","9,170,353","9,170,341","9,170,341","-9,170,605"
9,202504,Postpaid,B2S010602,Postpaid SubBase B2C,2025-10-27 06:22:35,20250430,0,"14,301,830","14,295,889","14,295,878","14,295,878","-14,301,830"


In [16]:
''' Postpaid Reported SubBase B2C '''

v_metric_list = [
    'B2S010600' #Postpaid Reported SubBase B2C
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    ]

postpaid_report_sub_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_report_sub_b2c_df = postpaid_report_sub_b2c_df.loc[postpaid_report_sub_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_report_sub_b2c_df = postpaid_report_sub_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_report_sub_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-10-27 06:22:35,20250131,0,"14,069,220","14,063,387","13,889,946","14,063,360","-14,069,220"
1,202501,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20250131,0,"5,033,939","5,028,342","4,854,928","5,028,342","-5,033,939"
2,202501,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,20250131,0,"9,035,281","9,035,045","9,035,018","9,035,018","-9,035,281"
3,202502,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-10-27 06:22:35,20250228,0,"14,068,955","14,063,110","14,063,094","14,063,094","-14,068,955"
4,202502,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20250228,0,"5,013,920","5,008,326","5,008,326","5,008,326","-5,013,920"
5,202502,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,20250228,0,"9,055,035","9,054,784","9,054,768","9,054,768","-9,055,035"
6,202503,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-10-27 06:22:35,20250331,0,"14,084,812","14,078,967","14,078,955","14,078,955","-14,084,812"
7,202503,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20250331,0,"4,997,992","4,992,399","4,992,399","4,992,399","-4,997,992"
8,202503,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,20250331,0,"9,086,820","9,086,568","9,086,556","9,086,556","-9,086,820"
9,202504,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-10-27 06:22:35,20250430,0,"14,100,846","14,094,971","14,094,960","14,094,960","-14,100,846"


### TOL

In [17]:
''' FTTx SubBase '''

v_metric_list = ['TB3S020604'] #FTTx SubBase

fttx_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_subbase_df = fttx_subbase_df.loc[fttx_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_subbase_df = fttx_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20250131,0,"3,280,710","3,280,625","3,280,625","3,280,625","-3,280,710"
1,202502,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20250228,0,"3,290,380","3,290,298","3,290,298","3,290,298","-3,290,380"
2,202503,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20250331,0,"3,305,177","3,305,095","3,305,095","3,305,095","-3,305,177"
3,202504,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20250430,0,"3,315,573","3,315,491","3,315,491","3,315,491","-3,315,573"
4,202505,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20250531,0,"3,294,820","3,294,739","3,294,739","3,294,739","-3,294,820"
5,202506,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20250630,0,"3,304,012","3,303,931","3,303,931","3,303,931","-3,304,012"
6,202507,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20250731,0,"3,309,412","3,309,333","3,309,333","3,309,333","-3,309,412"
7,202508,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20250831,0,"3,314,584","3,314,505","3,314,505","3,314,505","-3,314,584"
8,202509,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20250930,0,"3,318,426","3,318,347","3,318,347","3,318,347","-3,318,426"
9,202510,TOL,TB3S020604,FTTx SubBase,2025-10-27 06:22:35,20251024,0,"3,321,208","3,321,129","3,321,129","3,321,129","-3,321,208"


In [18]:
''' FTTx Reported SubBase '''

v_metric_list = ['TB3S000600'] #FTTx Reported SubBase

fttx_report_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_report_sub_df = fttx_report_sub_df.loc[fttx_report_sub_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_report_sub_df = fttx_report_sub_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_report_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20250131,0,"3,163,192","3,163,109","3,163,109","3,163,109","-3,163,192"
1,202502,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20250228,0,"3,172,770","3,172,690","3,172,690","3,172,690","-3,172,770"
2,202503,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20250331,0,"3,187,717","3,187,637","3,187,637","3,187,637","-3,187,717"
3,202504,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20250430,0,"3,191,503","3,191,424","3,191,424","3,191,424","-3,191,503"
4,202505,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20250531,0,"3,203,127","3,203,048","3,203,048","3,203,048","-3,203,127"
5,202506,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20250630,0,"3,207,941","3,207,862","3,207,862","3,207,862","-3,207,941"
6,202507,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20250731,0,"3,212,778","3,212,699","3,212,699","3,212,699","-3,212,778"
7,202508,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20250831,0,"3,218,435","3,218,356","3,218,356","3,218,356","-3,218,435"
8,202509,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20250930,0,"3,225,844","3,225,765","3,225,765","3,225,765","-3,225,844"
9,202510,TOL,TB3S000600,FTTx Reported SubBase,2025-10-27 06:22:35,20251024,0,"3,232,773","3,232,694","3,232,694","3,232,694","-3,232,773"


### TVS

In [19]:
''' TVS Active Subs '''

v_metric_list = ['TB4S000500'] #TVS Active Subs

tvs_active_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
tvs_active_sub_df = tvs_active_sub_df.loc[tvs_active_sub_df['TM_KEY_MTH']>=v_tm_key_mth]
tvs_active_sub_df = tvs_active_sub_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
tvs_active_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20250131,"1,170,444","1,170,444","1,170,444","1,170,444","1,170,444",0
1,202502,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20250228,"1,163,299","1,163,299","1,163,299","1,163,299","1,163,299",0
2,202503,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20250331,"1,153,137","1,153,137","1,153,137","1,153,137","1,153,137",0
3,202504,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20250430,"1,141,576","1,141,576","1,141,576","1,141,576","1,141,576",0
4,202505,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20250531,"1,129,379","1,129,379","1,129,379","1,129,379","1,129,379",0
5,202506,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20250630,"1,112,829","1,112,829","1,112,829","1,112,829","1,112,829",0
6,202507,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20250731,"1,098,044","1,098,044","1,098,044","1,098,044","1,098,044",0
7,202508,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20250831,"1,082,331","1,082,331","1,082,331","1,082,331","1,082,331",0
8,202509,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20250930,"1,071,847","1,071,847","1,071,847","1,071,847","1,071,847",0
9,202510,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,20251025,"1,064,471","1,064,471","1,064,471","1,064,471","1,064,471",0


## ** Current Issue

In [20]:
''' Monthly : Prepaid Active Caller 30D Rolling : TMH (Jan-Apr 2024) '''

v_metric_cd = 'TB1S000600'

issue_pre_actv_call_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_pre_actv_call_df = issue_pre_actv_call_df.loc[issue_pre_actv_call_df['TM_KEY_MTH']>=202401]
issue_pre_actv_call_df = issue_pre_actv_call_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_pre_actv_call_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202405,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-09-22 06:15:44,20240531,"17,391,242","17,391,242","17,190,679","17,190,679","17,190,679",0
1,202406,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-09-22 06:15:44,20240630,"17,313,488","17,313,488","17,219,683","17,219,683","17,219,683",0
2,202407,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-09-22 06:15:44,20240731,"17,342,927","17,342,927","17,323,115","17,323,115","17,323,115",0
3,202408,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-09-22 06:15:44,20240831,"17,247,805","17,247,805","17,131,822","17,131,822","17,131,822",0
4,202409,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-09-22 06:15:44,20240930,"17,190,424","17,190,424","17,178,862","17,178,862","17,178,862",0
5,202410,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-09-22 06:15:44,20241031,"17,195,820","17,195,820","17,183,235","17,183,235","17,183,235",0
6,202411,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-09-22 06:15:44,20241130,"17,253,443","17,253,443","17,239,155","17,239,155","17,239,155",0
7,202412,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-09-22 06:15:44,20241231,"17,769,553","17,769,553","17,753,679","17,753,679","17,753,679",0
8,202501,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,20250131,"17,344,294","17,344,294","17,330,909","17,330,909","17,330,909",0
9,202502,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,20250228,"17,424,676","17,424,676","17,407,425","17,407,425","17,407,425",0


In [21]:
''' Daily : Prepaid Active Caller 30D Rolling : TMH '''

v_metict_cd = 'TB1S000600'
# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

issue_daily_pre_actv_call_tmh_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_pre_actv_call_tmh_df = issue_daily_pre_actv_call_tmh_df.loc[issue_daily_pre_actv_call_tmh_df['METRIC_CD']==v_metict_cd]
issue_daily_pre_actv_call_tmh_df = issue_daily_pre_actv_call_tmh_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_pre_actv_call_tmh_df_display = issue_daily_pre_actv_call_tmh_df.copy()
mod_col_list = issue_daily_pre_actv_call_tmh_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_pre_actv_call_tmh_df_display[col] = issue_daily_pre_actv_call_tmh_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_pre_actv_call_tmh_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202510,20251026,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
1,202510,20251025,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,"16,025,537","16,010,916","16,010,916","16,010,916"
2,202510,20251024,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,"16,031,660","16,017,087","16,017,087","16,017,087"
3,202510,20251023,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,"16,030,226","16,015,771","16,015,771","16,015,771"
4,202510,20251022,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,"16,028,672","16,014,223","16,014,223","16,014,223"
5,202510,20251021,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,"16,023,578","16,009,248","16,009,248","16,009,248"
6,202510,20251020,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,"16,019,580","16,005,172","16,005,172","16,005,172"
7,202510,20251019,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,"15,999,259","15,984,673","15,984,673","15,984,673"
8,202510,20251018,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,"15,967,596","15,953,297","15,953,297","15,953,297"
9,202510,20251017,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-10-27 06:22:35,"15,974,402","15,960,085","15,960,085","15,960,085"


In [22]:
''' Monthly : Postpaid Reported SubBase B2C (Jan-May 2024) '''

v_metric_cd = 'B2S010600'

issue_postpaid_report_sub_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_report_sub_b2c_df = issue_postpaid_report_sub_b2c_df.loc[issue_postpaid_report_sub_b2c_df['TM_KEY_MTH']>=202401]
issue_postpaid_report_sub_b2c_df = issue_postpaid_report_sub_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_report_sub_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202406,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-09-22 06:15:44,20240630,0,"14,226,167","14,212,860","14,034,154","14,212,850","-14,226,167"
1,202407,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-09-22 06:15:44,20240731,0,"14,209,033","14,195,782","14,018,425","14,195,772","-14,209,033"
2,202408,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-09-22 06:15:44,20240831,0,"14,190,431","14,177,201","13,999,593","14,177,191","-14,190,431"
3,202409,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-09-22 06:15:44,20240930,0,"14,186,615","14,173,403","13,996,349","14,173,393","-14,186,615"
4,202410,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-09-22 06:15:44,20241031,0,"14,179,406","14,166,448","13,990,677","14,166,438","-14,179,406"
5,202411,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-09-22 06:15:44,20241130,0,"14,160,522","14,154,713","13,979,080","14,154,703","-14,160,522"
6,202412,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-09-22 06:15:44,20241231,0,"14,158,698","14,152,890","13,977,330","14,152,880","-14,158,698"
7,202501,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-10-27 06:22:35,20250131,0,"14,069,220","14,063,387","13,889,946","14,063,360","-14,069,220"
8,202502,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-10-27 06:22:35,20250228,0,"14,068,955","14,063,110","14,063,094","14,063,094","-14,068,955"
9,202503,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-10-27 06:22:35,20250331,0,"14,084,812","14,078,967","14,078,955","14,078,955","-14,084,812"


In [23]:
''' Monthly : Postpaid Reported SubBase B2C : TMH (Jan-May 2024) '''

v_metric_cd = 'TB2S010600'

issue_postpaid_report_sub_b2c_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_report_sub_b2c_tmh_df = issue_postpaid_report_sub_b2c_tmh_df.loc[issue_postpaid_report_sub_b2c_tmh_df['TM_KEY_MTH']>=202401]
issue_postpaid_report_sub_b2c_tmh_df = issue_postpaid_report_sub_b2c_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_report_sub_b2c_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202402,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240229,0,"8,998,482","8,998,152","8,998,139","8,998,139","-8,998,482"
1,202403,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240331,0,"8,994,853","8,987,179","8,987,169","8,987,169","-8,994,853"
2,202404,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240430,0,"8,996,646","8,989,810","8,989,800","8,989,800","-8,996,646"
3,202405,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240531,0,"9,023,324","9,009,568","9,009,558","9,009,558","-9,023,324"
4,202406,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240630,0,"9,042,184","9,041,896","9,041,886","9,041,886","-9,042,184"
5,202407,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240731,0,"9,052,569","9,052,314","9,052,304","9,052,304","-9,052,569"
6,202408,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240831,0,"9,061,002","9,060,761","9,060,751","9,060,751","-9,061,002"
7,202409,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20240930,0,"9,068,470","9,068,233","9,068,223","9,068,223","-9,068,470"
8,202410,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20241031,0,"9,080,143","9,079,912","9,079,902","9,079,902","-9,080,143"
9,202411,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-09-22 06:15:44,20241130,0,"9,088,656","9,088,441","9,088,431","9,088,431","-9,088,656"


In [24]:
''' Daily : Postpaid Reported SubBase B2C : TMH '''

v_metict_cd = 'TB2S010600'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_post_rpt_sb_b2c_tmh_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_rpt_sb_b2c_tmh_df = issue_daily_post_rpt_sb_b2c_tmh_df.loc[issue_daily_post_rpt_sb_b2c_tmh_df['METRIC_CD']==v_metict_cd]
issue_daily_post_rpt_sb_b2c_tmh_df = issue_daily_post_rpt_sb_b2c_tmh_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_rpt_sb_b2c_tmh_df_display = issue_daily_post_rpt_sb_b2c_tmh_df.copy()
mod_col_list = issue_daily_post_rpt_sb_b2c_tmh_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_rpt_sb_b2c_tmh_df_display[col] = issue_daily_post_rpt_sb_b2c_tmh_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_rpt_sb_b2c_tmh_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202510,20251026,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,nan,nan,nan,nan
1,202510,20251025,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,"9,051,017","9,050,779","9,050,771","9,050,771"
2,202510,20251024,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,"9,060,822","9,060,582","9,060,574","9,060,574"
3,202510,20251023,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,"9,058,101","9,057,862","9,057,854","9,057,854"
4,202510,20251022,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,"9,055,329","9,055,092","9,055,084","9,055,084"
5,202510,20251021,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,"9,053,028","9,052,792","9,052,784","9,052,784"
6,202510,20251020,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,"9,059,147","9,058,912","9,058,904","9,058,904"
7,202510,20251019,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,"9,055,687","9,055,452","9,055,444","9,055,444"
8,202510,20251018,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,"9,052,205","9,051,970","9,051,962","9,051,962"
9,202510,20251017,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-10-27 06:22:35,"9,057,982","9,052,745","9,052,736","9,052,736"


In [25]:
''' Monthly : Postpaid Reported SubBase B2C : DTAC (Jan-May 2024) '''

v_metric_cd = 'DB2S010600'

issue_postpaid_report_sub_b2c_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_report_sub_b2c_dtac_df = issue_postpaid_report_sub_b2c_dtac_df.loc[issue_postpaid_report_sub_b2c_dtac_df['TM_KEY_MTH']>=202401]
issue_postpaid_report_sub_b2c_dtac_df = issue_postpaid_report_sub_b2c_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_report_sub_b2c_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202406,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20240630,0,"5,183,983","5,170,964","4,992,268","5,170,964","-5,183,983"
1,202407,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20240731,0,"5,156,464","5,143,468","4,966,121","5,143,468","-5,156,464"
2,202408,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20240831,0,"5,129,429","5,116,440","4,938,842","5,116,440","-5,129,429"
3,202409,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20240930,0,"5,118,145","5,105,170","4,928,126","5,105,170","-5,118,145"
4,202410,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20241031,0,"5,099,263","5,086,536","4,910,775","5,086,536","-5,099,263"
5,202411,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20241130,0,"5,071,866","5,066,272","4,890,649","5,066,272","-5,071,866"
6,202412,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-09-22 06:15:44,20241231,0,"5,055,973","5,050,376","4,874,826","5,050,376","-5,055,973"
7,202501,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20250131,0,"5,033,939","5,028,342","4,854,928","5,028,342","-5,033,939"
8,202502,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20250228,0,"5,013,920","5,008,326","5,008,326","5,008,326","-5,013,920"
9,202503,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,20250331,0,"4,997,992","4,992,399","4,992,399","4,992,399","-4,997,992"


In [26]:
''' Daily : Postpaid Reported SubBase B2C : DTAC '''

v_metict_cd = 'DB2S010600'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_post_rpt_sb_b2c_dtac_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_rpt_sb_b2c_dtac_df = issue_daily_post_rpt_sb_b2c_dtac_df.loc[issue_daily_post_rpt_sb_b2c_dtac_df['METRIC_CD']==v_metict_cd]
issue_daily_post_rpt_sb_b2c_dtac_df = issue_daily_post_rpt_sb_b2c_dtac_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_rpt_sb_b2c_dtac_df_display = issue_daily_post_rpt_sb_b2c_dtac_df.copy()
mod_col_list = issue_daily_post_rpt_sb_b2c_dtac_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_rpt_sb_b2c_dtac_df_display[col] = issue_daily_post_rpt_sb_b2c_dtac_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_rpt_sb_b2c_dtac_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202510,20251026,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,nan,nan,nan,nan
1,202510,20251025,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,nan,nan,nan,nan
2,202510,20251024,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,"4,879,958","4,874,374","4,874,374","4,874,374"
3,202510,20251023,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,"4,878,684","4,873,100","4,873,100","4,873,100"
4,202510,20251022,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,"4,877,200","4,871,616","4,871,616","4,871,616"
5,202510,20251021,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,"4,876,185","4,870,601","4,870,601","4,870,601"
6,202510,20251020,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,"4,875,743","4,870,159","4,870,159","4,870,159"
7,202510,20251019,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,"4,874,016","4,868,432","4,868,432","4,868,432"
8,202510,20251018,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,"4,872,801","4,867,217","4,867,217","4,867,217"
9,202510,20251017,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-10-27 06:22:35,"4,880,964","4,875,379","4,875,379","4,875,379"


In [27]:
''' Monthly : TVS Active Subs '''

v_metict_cd = 'TB4S000500'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_tvs_active_sub_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_tvs_active_sub_df = issue_tvs_active_sub_df.loc[issue_tvs_active_sub_df['METRIC_CD']==v_metict_cd]
issue_tvs_active_sub_df = issue_tvs_active_sub_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_tvs_active_sub_df_display = issue_tvs_active_sub_df.copy()
mod_col_list = issue_tvs_active_sub_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_tvs_active_sub_df_display[col] = issue_tvs_active_sub_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_tvs_active_sub_df_display


,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202510,20251026,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,nan,nan,nan,nan
1,202510,20251025,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,"1,064,471","1,064,471","1,064,471","1,064,471"
2,202510,20251024,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,"1,064,809","1,064,809","1,064,809","1,064,809"
3,202510,20251023,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,"1,065,047","1,065,047","1,065,047","1,065,047"
4,202510,20251022,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,"1,065,539","1,065,539","1,065,539","1,065,539"
5,202510,20251021,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,"1,065,064","1,065,064","1,065,064","1,065,064"
6,202510,20251020,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,"1,065,064","1,065,064","1,065,064","1,065,064"
7,202510,20251019,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,"1,065,355","1,065,355","1,065,355","1,065,355"
8,202510,20251018,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,"1,065,931","1,065,931","1,065,931","1,065,931"
9,202510,20251017,TVS,TB4S000500,TVS Active Subs,2025-10-27 06:22:35,"1,066,108","1,066,108","1,066,108","1,066,108"
